# Evaluate MobileNet V2

Import relevant libraries

In [4]:
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from wandb.keras import WandbCallback

import logging
import pandas as pd
import tensorflow as tf
import wandb

In [2]:
wandb.login()

wandb: Currently logged in as: burntice (use `wandb login --relogin` to force relogin)


True

In [5]:
# Initialise Weights and Biases API
# Hyperparameters
run = wandb.init(
    name='resnet50_v2',
    project='cz4042',
    config={
        'batch_size': 128,
        'epochs': 20,
        'seed': 0,
        'optimizer': 'sgd',
        'loss_function': 'binary_crossentropy',
        'metrics': ['accuracy'],
    },
)

config = wandb.config

logger = logging.getLogger('wandb')
logger.setLevel(logging.ERROR)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Prevent GPU memory usage from exceeding GPU memory

In [7]:
# reference: https://github.com/tensorflow/tensorflow/issues/25160#issuecomment-643703167
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 4)],
        )
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), 'Physical GPUs,', len(logical_gpus), 'Logical GPUs')
    except RuntimeError as error:
        # Virtual devices must be set before GPUs have been initialized
        print(error)
else:
    print('No GPU detected!')

1 Physical GPUs, 1 Logical GPUs


Load data

In [8]:
data_file_path = '/home/burntice/0_repositories/CZ4042/project/data/aligned_gender.csv'
data_dir_path = '/home/burntice/3_data/Adience/'

dataframe = pd.read_csv(data_file_path, sep='\t', names=['datadir', 'gender'])

In [9]:
# confirm dataframe is ok
dataframe.head()

,datadir,gender
0,datadir gender,NaN
1,aligned/30601258@N03/landmark_aligned_face.2.1...,1.0
2,aligned/30601258@N03/landmark_aligned_face.3.1...,1.0
3,aligned/30601258@N03/landmark_aligned_face.2.1...,1.0
4,aligned/30601258@N03/landmark_aligned_face.4.1...,0.0


In [10]:
# check number of data in dataframe
dataframe.shape

(12195, 2)

In [11]:
# check percentage of gender in data
dataframe['gender'].value_counts() / dataframe.shape[0] * 100

0.0    52.660927
1.0    47.330873
Name: gender, dtype: float64

Not very imbalanced; good.

In [12]:
x_train, x_test, y_train, y_test = train_test_split(
    dataframe['datadir'],
    dataframe['gender'],
    test_size=0.2,
    random_state=config.seed,
)

In [13]:
# checking to make sure everything is alright
x_train.head()

6753     aligned/7636528@N03/landmark_aligned_face.1052...
8876     aligned/113650443@N02/landmark_aligned_face.14...
9269     aligned/48647239@N03/landmark_aligned_face.151...
1022     aligned/37303189@N08/landmark_aligned_face.84....
10913    aligned/10280355@N07/landmark_aligned_face.189...
Name: datadir, dtype: object

In [14]:
train_df = pd.concat([x_train, y_train], axis=1)
test_df = pd.concat([x_test, y_test], axis=1)

In [15]:
train_df.shape

(9756, 2)

In [16]:
train_df['gender'].value_counts() / train_df.shape[0] * 100

0.0    52.97253
1.0    47.01722
Name: gender, dtype: float64

In [18]:
# Load images into keras image generator 
datagen_train = ImageDataGenerator(rescale=1./255)
datagen_val = ImageDataGenerator(rescale=1./255)

# For train generator
train_generator = datagen_train.flow_from_dataframe(
    dataframe=train_df,
    directory=data_dir_path,
    x_col='datadir',
    y_col='gender',
    batch_size=config.batch_size,
    seed=config.seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

# For test generator 
val_generator = datagen_val.flow_from_dataframe(
    dataframe = test_df,
    directory=data_dir_path,
    x_col='datadir',
    y_col='gender',
    batch_size=config.batch_size,
    seed=config.seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

Found 9755 validated image filenames.
Found 2439 validated image filenames.


In [19]:
model = tf.keras.models.Sequential([
    tf.keras.applications.MobileNetV2(include_top=False, pooling='avg', weights='imagenet'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='softmax'),
])

model.layers[0].trainable = False
model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 1280)              2257984   
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1280)              5120      
_________________________________________________________________
dense (Dense)                (None, 1024)              1311744   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
___________________________________________________________

In [20]:
# Compile model 
model.compile(
    optimizer=config.optimizer,
    loss=config.loss_function,
    metrics=config.metrics,
)

Define callback for saving weights while training

In [21]:
checkpoint_dir_path = '/home/burntice/0_repositories/CZ4042/project/checkpoints/'
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=(checkpoint_dir_path + 'cp-{epoch:02d}-{val_loss:.2f}.hdf5'),
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss'
)

In [22]:
model.fit(
    train_generator,
    epochs=config.epochs,
    validation_data=val_generator,
    callbacks=[WandbCallback(), cp_callback],
)

Epoch 1/20
  12353/Unknown - 14368s 1s/step - loss: 8.0784 - accuracy: 0.4702

Let Weights and Biases know that this run is complete.

In [ ]:
run.finish()